In [66]:
import torch
import pandas as pd
from pathlib import Path
from datetime import datetime
from torch.utils.data import Dataset
from random import sample
import os
import numpy as np

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [9]:
model_path = 'embeding_models/embeding_v1_16_dim.pt'
model = torch.jit.load(model_path).encoder.to(device)

In [5]:
BASE_DATE = datetime.strptime("2025-01-03", '%Y-%m-%d').timestamp()
print(BASE_DATE)

1735858800.0


In [18]:
tracks_raw_data = pd.read_json(Path('../data_v2/tracks_artists.jsonl'), lines=True)


# NORMALIZE DATES
tracks_raw_data["release_date"] = (pd.to_datetime(tracks_raw_data["release_date"], format='mixed').apply(lambda x: x.timestamp())).div(BASE_DATE)

# NORMALIZE DURATION
tracks_raw_data["duration_ms"] = tracks_raw_data["duration_ms"].div(tracks_raw_data["duration_ms"].max())

# NORMALIZE TEMPO
tracks_raw_data["tempo"] = tracks_raw_data["tempo"].div(tracks_raw_data["tempo"].max())

# EXPLICITE ENCODING
tracks_raw_data["explicit"] = tracks_raw_data["explicit"].apply(lambda x: [0, 1] if x else [1, 0])

# processs ARTIST HASH

def postprocess_hash_to_list(x):
    str_x = str(x)
    if len(str_x) < 8:
        str_x = "0" * (8 - len(str_x)) + str_x
    return [int(x) for x in str_x]

tracks_raw_data["id_artist_hash"] = tracks_raw_data["id_artist_hash"].apply(postprocess_hash_to_list)



In [7]:
class TracksDataset(Dataset):
    def __init__(self, tracks_data: pd.DataFrame):
        self.data = tracks_data

    def __len__(self):
        return len(self.data)

    def get_item(self, idx):
        return self.data.iloc[idx].values

    def __getitem__(self, idx):
        unpacked_data = []
        for data in self.data.iloc[idx].drop("id_track").values:
            if type(data) != list:
                unpacked_data.append(data)
            else:
                unpacked_data += data
        return torch.Tensor(unpacked_data)

In [19]:
embedings_per_id = pd.DataFrame()
embedings_per_id["id_track"] = tracks_raw_data["id_track"]
embedings_per_id["embeding"] = [model(torch.Tensor(x).to(device)).detach().cpu().numpy() for x in TracksDataset(tracks_raw_data)]

In [27]:
test_intigers = [687, 852, 528, 562, 426, 1092, 171, 250, 223, 265, 981, 607, 738, 1020, 510, 899, 596, 1047, 826, 669, 923, 905, 1063, 139, 1031]

In [37]:
files_to_load = [file for file in os.listdir("../data_v2/sessions") if int(file.split(".")[0].split("_")[-1]) not in test_intigers]

raw_sessions_data = pd.concat([pd.read_json(Path(f'../data_v2/sessions/{file}'), lines=True) for file in files_to_load])
print(raw_sessions_data.head())

   session_id               timestamp  user_id                track_id  \
0       89426 2024-09-17 10:51:23.000      260  1xPec5BN0Zxv77zrWKq43S   
1       89427 2023-05-20 00:35:36.227      260  7s0lDK7y3XLmI7tcsRAbW0   
2       89427 2023-05-20 00:38:48.054      260  48lQegoLqGAzaRLnMwK0mO   
3       89427 2023-05-20 00:43:30.802      260  48lQegoLqGAzaRLnMwK0mO   
4       89427 2023-05-20 00:44:59.387      260  4usVYcPlxRgRet6YashdCJ   

  event_type  
0       play  
1       play  
2       play  
3       like  
4       play  


In [125]:
merged_sesions_data = raw_sessions_data.merge(embedings_per_id, left_on="track_id", right_on="id_track").drop("id_track", axis=1)
merged_sesions_data["timestamp"] = (pd.to_datetime(merged_sesions_data["timestamp"], format='mixed').apply(lambda x: x.timestamp())).div(BASE_DATE)
merged_sesions_data = pd.get_dummies(merged_sesions_data, columns=["event_type"], dtype = int)

merged_sesions_data = [pd.DataFrame(y) for _, y in merged_sesions_data.groupby('session_id', as_index=False) if len(y) > 1]

In [ ]:
print(merged_sesions_data[0])

a


         session_id  timestamp  user_id                track_id  \
1185842         124   0.993923      101  1ZAkJE2vi1wbo7tyvgWuXN   
1185843         124   0.993923      101  6YpSiNQN8pVzJMOX2fXGHm   
1185844         124   0.993924      101  6YpSiNQN8pVzJMOX2fXGHm   

                                                  embeding  event_type_like  \
1185842  [4.7692018, 0.50016445, 1.8391547, -4.9334793,...                0   
1185843  [3.1930203, -1.8490876, 6.483166, -4.8640914, ...                0   
1185844  [3.1930203, -1.8490876, 6.483166, -4.8640914, ...                1   

         event_type_play  event_type_skip  
1185842                1                0  
1185843                1                0  
1185844                0                0  


In [128]:
class CustomUserSesionsDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __len__(self):
        return len(self.data)

    def get_item(self, idx):
        return self.data.iloc[idx].values

    def __getitem__(self, idx):
        sessions = []
        for session in self.data[idx].drop("track_id", axis=1).drop("session_id", axis=1).drop("user_id", axis=1).values:
            unpacked_data = np.array([])
            for data in session:
                unpacked_data = np.append(unpacked_data, data)
            sessions.append(torch.tensor(unpacked_data))
        return sessions

In [129]:
user_sesions_dataset = CustomUserSesionsDataset(merged_sesions_data)